In [29]:
import os
import sys; sys.path.append(os.getcwd()+'/src')
from importlib import reload
import class_handler as classes; reload(classes)
import numpy as np
import pandas as pd

## Generate data to play with

In [3]:
#from sklearn.datasets import load_breast_cancer
#x, y = load_breast_cancer(return_X_y=True)

import sklearn
x, y = sklearn.datasets.load_digits(return_X_y=True)

print(x.shape, y.shape)

(1797, 64) (1797,)


In [4]:
# Lets split into train and test
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1437, 64) (1437,) (360, 64) (360,)


In [34]:
from sklearn.metrics import log_loss
import numpy as np
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

y_true = [0, 1, 2, 3, 1]
y_pred = [1, 1, 2, 3, 0]
y_pred = lb.fit_transform(np.array(y_pred))
print(y_pred)
log_loss(y_true, y_pred)

[[0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]]


13.815510557964274

In [44]:
y_pred = np.array([1, 1, 2, 3, 0])
len(y_pred.shape)

1

In [33]:
from base import EnsembleBaseClassifier
module_name = 'adaboost'
algorithm_name = 'MetaAdaBoostClassifierAlgorithm'
from importlib import reload, import_module               

try:
    module = import_module(module_name)
except:
    raise ValueError("Could not import module '%s'" % module_name)                
algorithm = getattr(module, algorithm_name)

# check if 'algorithm' is a subclass of 'EnsembleBaseClassifier' 
# and set property 'base_estimator' if so
if issubclass(algorithm, EnsembleBaseClassifier):
    print('catched!!')
    instance = algorithm(base_estimator=self.base_estimator)
else:
    instance = algorithm()

catched!!


NameError: name 'self' is not defined

In [38]:
import adaboost; reload(adaboost)
from adaboost import MetaAdaBoostClassifierAlgorithm 
superclass = MetaAdaBoostClassifierAlgorithm().__class__.__base__.__base__
superclass

object

## Load using new API

In [43]:
# Import classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import class_handler as metawrapper; reload(metawrapper)
clfs = metawrapper.MetaWrapperClassifier(verbose=1, method='complete', base_estimator=KNeighborsClassifier())
for n, clf in clfs.clf:
    try:
        print(clf.base_estimator)
    except:
        print(clf.name)
clf.estimator.get_params()

Initialized classifiers:
	adaboost
	nearestneigbors
	logreg
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
nearestneigbors
logreg


{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [ ]:
clfs.fit_classifiers(X_train, y_train, n_jobs=2)

In [ ]:
y_train_pred = clfs.predict_classifiers(X_train)

In [ ]:
# Evaulate performance on training data
print("Training scores:")
train_scores = clfs.classifier_performance(y_train_pred, y_train, metric='accuracy')
# Evaluate overfitting
print("Test scores:")
test_scores = clfs.classifier_performance(clfs.predict_classifiers(X_test), y_test, metric='accuracy')
# AdaBoost clearly overfits this dataset while LogiReg and KNN seem ok

In [ ]:
import visualize; reload(visualize)
from visualize import Visualizer
# This method expects a list of 2-tuples
vis = Visualizer()
vis.show_performance([tuple([x,y]) for x,y,_ in test_scores], fig_size=(12,6))

In [ ]:
from numpy import random
ll = [('a',1),('b',2),('c',3)]
idx = np.random.choice(len(ll), 1, replace=False)
sample = []
for id in idx:
    sample.append(ll[id])
sample

In [ ]:
from sklearn.metrics import accuracy_score, log_loss

# Import classifiers
import class_handler as classes; reload(classes)
clfs = classes.Classifiers(verbose=1, method='complete')
np.random.seed(1)
optims = clfs.optimize_classifiers(X_train, y_train, n_iter=100, n_jobs=-1, random_state=15)

print()
for name, clf in optims:
    print("Training score: ", name, accuracy_score(clf.predict(X_train), y_train))
    print("Test score: ", name, accuracy_score(clf.predict(X_test), y_test)) 
    print()

In [ ]:
import class_handler as classes; reload(classes)
clfs = classes.Classifiers(verbose=1, method='chosen', clf=['logreg'])
# Assume now that you only want to tune C-parameter in LogisticRegression() classifier:
for name, classifier in clfs.clf:
    if isinstance(classifier.estimator, type(LogisticRegression())):
        print("Lorem Ipsum!!!!!!")
        for p in ['C', 'fit_intercept']:
            classifier.cv_params_to_tune.append(p)
        print( classifier.cv_params_to_tune )
        assert set(classifier.cv_params_to_tune).issubset(set(classifier.cv_params.keys())), "[%s]" % ", ".join(classifier.cv_params.keys())

# Now we can call optimize_classifiers 
for cv in [3,5,10]:
    np.random.seed(cv)
    print("cv = ",cv)
    optims = clfs.optimize_classifiers(X_train, y_train, n_iter=100, cv=cv, n_jobs=-1)
    print()
    for name, clf in optims:
        print("Training score: ", name, accuracy_score(clf.predict(X_train), y_train))
        print("Test score: ", name, accuracy_score(clf.predict(X_test), y_test)) 
        print()

## Base class for algorithms

### Testing with adaboost

In [ ]:
import adaboost; reload(adaboost)
from adaboost import MetaAdaBoostClassifierAlgorithm
ada = MetaAdaBoostClassifierAlgorithm(random_state=0, base_estimator='logreg')
print(ada.get_info())
ada.cv_param_dist = ada._param_dist_dict()
print(ada.cv_param_dist)

# specify parameters and distributions to sample from
from sklearn.model_selection import RandomizedSearchCV
param_dist = ada.cv_param_dist
estimator = ada.estimator

# IT SEEMS THAT THE LOGISTIC REGRESSION CLASSIFIER TENDS TO OVERFIT...SHOULD LOWER n_iter IF USING THAT ALGO.
grid_search = RandomizedSearchCV(estimator, param_distributions=param_dist, n_iter=1, scoring='neg_log_loss',
                       cv=10, n_jobs=-1, verbose=0, error_score=0, return_train_score=True)
grid_search.fit(X_train, y_train)
print("Best mean score: %.4f" % (grid_search.best_score_))
from sklearn.metrics import accuracy_score
print("Accuracy on test set: %.4f" % accuracy_score(y_test, grid_search.best_estimator_.predict(X_test))) 

grid_search.best_estimator_

### Testing with nearest neighbors algorithm

In [ ]:
import nearest_neighbors; reload(nearest_neighbors)
from nearest_neighbors import MetaKNearestNeighborClassifierAlgorithm
knn = MetaKNearestNeighborClassifierAlgorithm()
print(knn.get_info())
knn.cv_param_dist = knn._param_dist_dict()
print(type(knn.cv_param_dist))

# specify parameters and distributions to sample from
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
param_dist = knn.cv_param_dist
estimator = knn.estimator

In [ ]:
from pprint import pprint
pprint(param_dist.param_grid)
print("number of grids = %i" % len(param_dist.param_grid))

In [ ]:
results = []
for grid in param_dist.param_grid:
    grid_search = RandomizedSearchCV(estimator, param_distributions=grid, n_iter=2, scoring='accuracy',
                           cv=10, n_jobs=-1, verbose=0, error_score=0, return_train_score=True)
    #grid_search = GridSearchCV(estimator, param_grid=param_dist.param_grid, scoring='accuracy',
    #                       cv=10, n_jobs=-1, verbose=0, error_score=0, return_train_score=True)
    grid_search.fit(X_train, y_train)
    print("Best mean score: %.4f" % (grid_search.best_score_))
    from sklearn.metrics import accuracy_score
    print("Accuracy on test set: %.4f" % accuracy_score(y_test, grid_search.best_estimator_.predict(X_test))) 
    results.append((grid_search.best_score_, grid_search.best_estimator_))

print(results)

In [ ]:
# Import classifiers
clfs = classes.Classifiers(verbose=0, method='complete', num_clf=10)

# Fitting and tuning of parameters

We need a way of optimizing the hyper parameters of each selected algorithm. 

## Regular fitting (just train)

This is the basic building block: "fit_classifiers" loops over each algorithm and trains on data

In [ ]:
# Fit classifiers on data
clfs.fit_classifiers(X_train,y_train,1)

## Tune hyper parameters using cross validation

Scoring options:
* ['accuracy', 'adjusted_rand_score', 'average_precision', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc']

### Discrete and categorical variables (using scipy.stats)

In [ ]:
from sklearn.metrics import make_scorer
scorer = make_scorer(log_loss, greater_is_better=False)
scorer

In [ ]:
def neg_log_loss(y_true, y_pred):
    return -1.0 * log_loss(y_true, y_pred)
scorer = make_scorer(neg_log_loss)
scorer

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.ensemble import RandomForestClassifier
# specify parameters and distributions to sample from
param_dist = {"max_depth": [1, None],
              "max_features": sp_randint(1, 50),
              "min_samples_split": sp_randint(1, 50),
              "min_samples_leaf": sp_randint(1, 50),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"],
              "n_estimators": [100, 200, 300]} 
estimator = RandomForestClassifier()
grid_search = RandomizedSearchCV(estimator, param_distributions=param_dist, n_iter=25, scoring='accuracy',
                       cv=5, n_jobs=-1, verbose=0, error_score=0, return_train_score=True)
grid_search.fit(X_train, y_train)
#pd.DataFrame(grid_search.cv_results_).head()
print("Best mean score: %.4f" % (grid_search.best_score_))

from sklearn.metrics import accuracy_score
print("Accuracy on test set: %.4f" % accuracy_score(y_test, grid_search.best_estimator_.predict(X_test)))

### Continuous variable (using scipy.stats)

In [ ]:
from scipy.stats import uniform, gamma, lognorm
from sklearn.linear_model import LogisticRegression
param_dist = {"C": uniform(loc=0., scale=200.),
              "fit_intercept": [True, False],
              "penalty": ['l1', 'l2'],
              "max_iter": [50, 100, 150]}
estimator = LogisticRegression()
grid_search = RandomizedSearchCV(estimator, param_distributions=param_dist, n_iter=200, scoring='accuracy',
                       cv=10, n_jobs=-1, verbose=0, error_score=0, return_train_score=True)
grid_search.fit(X_train, y_train)
print("Best mean score: %.4f" % (grid_search.best_score_))
print(grid_search.best_params_)
from sklearn.metrics import accuracy_score
print("Accuracy on test set: %.4f" % accuracy_score(y_test, grid_search.best_estimator_.predict(X_test)))

In [ ]:
# Predict on data (returns a list of (name, prediction) tuples for each classifier in repository)
preds = clfs.predict_classifiers(X_train)
clfs.verbose = 1 # Report progress if =1, else be quiet if =0.
# How well are we doing?
scores = clfs.classifier_performance(preds, y_train, metric='accuracy')

In [ ]:
preds = clfs.predict_classifiers(X_test)
clfs.verbose = 1 # Report progress if =1, else be quiet if =0.
# How well are we doing?
scores = clfs.classifier_performance(preds, y_test, metric='accuracy')

## Check correlations

In [ ]:
# Create an instance "correlation checking" classes
import class_handler as classes; reload(classes)
check = classes.CheckClassifierCorrelation(prediction_type='c_binary')
# Get names and correlation matrix
names_, corr_ = check.compute_correlation_matrix(preds)
# Plot correlation matrix (output depends on if plotting inline or not)
fig = check.plot_correlation_matrix(names_, corr_, fig_size=(16,10), rot=20, font_scale=1.1, file=os.getcwd()+"/corr.png")

In [ ]:
%matplotlib inline
fig